# Инструмент продавца

## Придумать и реализовать прототип аналитического инструмента для продавцов, который поможет им продвигать свои товары на маркетплейсе

$\large{В\: датасете\: {\color{Purple}{WB\_hack \_3 .csv}}\: содержатся\: следующие\: данные:}$

* <span style="color:green"><b>utc_event_time</b></span> – время срабатывание события
* <span style="color:green"><b>utc_event_date</b></span> – дата события
* <span style="color:green"><b>user_id</b></span> – id клиента
* <span style="color:green"><b>city_name</b></span> – город
* <span style="color:green"><b>ecom_event_action</b></span> – действие с товаром
* <span style="color:green"><b>ecom_id</b></span> – id товара
* <span style="color:green"><b>ecom_brand</b></span> – название бренда
* <span style="color:green"><b>ecom_variant</b></span> – вариация товара ( красный/синий, 23/24, 1метр/2метра)
* <span style="color:green"><b>ecom_currency</b></span> – валюта товара
* <span style="color:green"><b>ecom_price100</b></span> – цена товара в копейках
* <span style="color:green"><b>ecom_qty</b></span> – количество товара
* <span style="color:green"><b>ecom_grand_total100</b></span> – сумма товара ( цена*кол-во)
* <span style="color:green"><b>os_manufacturer</b></span> – производитель устройства
* <span style="color:green"><b>device_type</b></span> – тип девайса
* <span style="color:green"><b>traffic_src_kind</b></span> – тип трафика
* <span style="color:green"><b>app_version</b></span> – версия приложения
* <span style="color:green"><b>net_type</b></span> – тип сети
* <span style="color:green"><b>locale</b></span> – локаль

In [374]:
# импортируем библиотеки

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [375]:
# читаем файл

wild_cut = pd.read_csv('wild_cut.csv')

### 1. Чистим данные 

In [376]:
# данные в столбцах ecom_event_action, ecom_id, ecom_brand, ecom_variant, ecom_currency
# нужно преобразовать, а именно убрать вначале и в конце строки символы "['",  "']"

# задаем список названий колонок, в которых будем преобразовывать данные
columns_with_bad_values = ['ecom_event_action', 'ecom_id', 'ecom_brand',
                           'ecom_variant', 'ecom_currency']

In [377]:
# с помощью регулярных выражений убираем лишнее из столбцов

wild_cut[columns_with_bad_values] = wild_cut[columns_with_bad_values] \
                              .apply(lambda x: x.str.extract(r'(\w+)', expand=False))

In [378]:
# как видим, все получилось

wild_cut.head()

,Unnamed: 0,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,[248500],[1],[248500],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,[75400],[1],[75400],samsung,smartphone,mobile,4.2.7000,3G,ru
2,2,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,[74500],[1],[74500],samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,[131900],[1],[131900],samsung,smartphone,mobile,4.3.1003,4G,ru
4,4,2021-10-21 19:43:45,2021-10-21,12954672620163632510,Москва,remove_from_cart,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,[1077100],[1],[1077100],samsung,smartphone,mobile,4.3.5000,Wi-Fi,ru


In [379]:
# делаем то же самое для столбцов ecom_price100, ecom_qty, ecom_grand_total100,
# только приводим их к типу int32

columns_to_int32 = ['ecom_price100', 'ecom_qty', 'ecom_grand_total100']

In [380]:
# пользуемся регулярным выражением

wild_cut[columns_to_int32] = wild_cut[columns_to_int32] \
                              .apply(lambda x: x.str.extract(r'(\d+)', expand=False)) \
                              .astype(dtype='int32')

In [381]:
# переведем цену товара и сумму товара в рубли

wild_cut['ecom_price100'] = wild_cut['ecom_price100'] / 100
wild_cut['ecom_grand_total100'] = wild_cut['ecom_grand_total100'] / 100

In [382]:
# переименуем колонки

wild_cut = wild_cut.rename(columns={'ecom_grand_total100': 'ecom_grand_total', 
                                    'ecom_price100': 'ecom_price'})

In [383]:
# посмотрим, какие типы данных у нас есть

wild_cut.dtypes

Unnamed: 0             int64
utc_event_time        object
utc_event_date        object
user_id               uint64
city_name             object
ecom_event_action     object
ecom_id               object
ecom_brand            object
ecom_variant          object
ecom_currency         object
ecom_price           float64
ecom_qty               int32
ecom_grand_total     float64
os_manufacturer       object
device_type           object
traffic_src_kind      object
app_version           object
net_type              object
locale                object
dtype: object

In [384]:
# здесь уместно перевести колонки utc_event_time и utc_event_date в 
# формат datetime

wild_cut[['utc_event_time']] = wild_cut[['utc_event_time']] \
                                .apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')

wild_cut[['utc_event_date']] = wild_cut[['utc_event_date']] \
                                .apply(pd.to_datetime, format='%Y-%m-%d')

# можно было просто написать pd.to_datetime(wild_cut.utc_event_time, 
#                                           format = '%Y-%m-%d %H:%M:%S'),
# но используем общую формулу для нескольких столбцов

In [385]:
# мы почистили данные, и можем с ними работать дальше

wild_cut.head()

,Unnamed: 0,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price,ecom_qty,ecom_grand_total,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,2485.0,1,2485.0,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,754.0,1,754.0,samsung,smartphone,mobile,4.2.7000,3G,ru
2,2,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,745.0,1,745.0,samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1319.0,1,1319.0,samsung,smartphone,mobile,4.3.1003,4G,ru
4,4,2021-10-21 19:43:45,2021-10-21,12954672620163632510,Москва,remove_from_cart,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,10771.0,1,10771.0,samsung,smartphone,mobile,4.3.5000,Wi-Fi,ru


In [386]:
wild_cut.nunique()

Unnamed: 0           2000
utc_event_time       1986
utc_event_date         50
user_id              1512
city_name             264
ecom_event_action       5
ecom_id                60
ecom_brand             51
ecom_variant           26
ecom_currency           1
ecom_price            359
ecom_qty                2
ecom_grand_total      360
os_manufacturer        32
device_type             1
traffic_src_kind        1
app_version            22
net_type                5
locale                  2
dtype: int64

### 2. Определимся с метриками

#### 2.1 Разделим товары по 3 группам:
#### I группа -> приносящие прибыль 80%
#### II группа -> приносящие прибыль 15%
#### III группа -> приносящие прибыль 5% 

In [387]:
# сгруппируем товары по ecom_id, 
# и разобьем по группам

all_sum_ecom = wild_cut.ecom_grand_total.sum()

our_percent = wild_cut.groupby('ecom_id') \
                      .agg({'ecom_grand_total': 'sum'}) \
                      .rename(columns={'ecom_grand_total': 'profit_percent'}) \
                      .sort_values(by='profit_percent',
                                   ascending=False) / all_sum_ecom * 100

In [388]:
our_percent.head()

,profit_percent
ecom_id,
B0270133170E1B886F4FA77B7E1F0EB9,31.256755
BDBD23C5359EF7B3F85763C032892B85,28.668720
8360E6D158AEBD26BBBE835C38F88748,27.157693
AC688D3B8E8D812E2B49496BE59286E3,3.270246
97F9B1135CF24F4CFFF4664435AEC2B9,1.790152


In [389]:
our_percent.shape

(60, 1)

In [390]:
# добавляем в списки товары каждой группы

group_1 = []
group_2 = []
group_3 = []

lim = 0
for ecom_id, percent in zip(our_percent.index, our_percent.profit_percent):
        if lim < 80:
            group_1.append(ecom_id)
            lim += percent
        elif lim < 95:
            group_2.append(ecom_id)
            lim += percent
        else:
            group_3.append(ecom_id)

In [391]:
# выводим продавцу наименования товаров:
# I группа -> приносит 80% прибыли;
# II группа -> приносит 15% прибыли;
# III группа -> приносит 5% прибыли -> рекомендуем от продажи
#                                      этих товаров отказаться
#                                      или пересмотреть их параметры
#                                      (описание, цвет и т.д.)

print('I группа: ', group_1, '\n\n',
      'II группа: ', group_2, '\n\n',
      'III группа: ', group_3, '\n\n')

I группа:  ['B0270133170E1B886F4FA77B7E1F0EB9', 'BDBD23C5359EF7B3F85763C032892B85', '8360E6D158AEBD26BBBE835C38F88748'] 

 II группа:  ['AC688D3B8E8D812E2B49496BE59286E3', '97F9B1135CF24F4CFFF4664435AEC2B9', 'ECD8847A3E389369BA25918DFF9B5B83', 'AF51FBD3B09A5BE324C76C524118BFD3', '6050B6B1F05ABDB4B33DA4BCEB46DE9B'] 

 III группа:  ['BF4304D1BE666DB38DF823DE35823D4D', '61F3488FEF9234C5B0E9B68CAF0AB51F', '7D9A98C6E8BF033613B74252D9D24B33', 'BEFC96A9D6DEA3A9817BC5E5157B0A1C', '5C8925190BE8C493517153B561BA21E7', 'A6FC1D41C2A36CD6ED1428BC40991F3A', '484FD293DAE723EB16AF9C7548C85ED9', '30DCD4FD8AB5E0ED4706DC0A19F5C7A9', '02C451A73FDD41CDD05F1E33E4BC60FC', 'B47F278C96726C7B5022C3007EB0A6C9', 'C0C9F2DC21904776545FFBB2FC3D646C', '67FC8BB094EC6205479255C1F8028E20', 'CA2AC4DF8A5C07E398DB8584A23D57F8', '2AA18144D676AC3D04B5250B416DEB64', '0F49D92CEA8C80166B0605ED3D1A8E5D', '6D88AAEEAB611F290D931ACCC12045CB', 'C1506A9C4FBD8AD89DA9FA0CC6616566', '6BDA30FE61747A5C2FA82F13843104A8', 'B3A955CA48D147C9C9

#### 2.2 Посчитаем конверсию для каждого товара
#### (купил / положил в корзину)

In [392]:
# сначала посмотрим действия, которые совершаются с товарами

wild_cut.ecom_event_action.unique()

# add_to_cart -> добавлено в корзину
# view_item -> просмотрен товар
# remove_from_cart -> удалено из корзины
# begin_checkout -> начато офрмление заказа
# purchase -> покупка

array(['add_to_cart', 'view_item', 'remove_from_cart', 'begin_checkout',
       'purchase'], dtype=object)

In [393]:
# отберем товары с ecom_event_action = ['add_to_cart', 'purchase']
# и сгруппируем их

product_conversion = wild_cut[[
                      'ecom_id',
                      'ecom_event_action',
                      'user_id']].query("ecom_event_action == ['add_to_cart','purchase']") \
                                 .groupby(['ecom_id', 'ecom_event_action']) \
                                 .agg({'user_id': 'count'}) \
                                 .rename(columns={'user_id': 'count_action'}) \
                                 .unstack(level='ecom_event_action', fill_value=0) \
                                 ['count_action'] \
                                 .reset_index() \
                                 .query('add_to_cart > 0')
        

In [394]:
product_conversion.head()

ecom_event_action,ecom_id,add_to_cart,purchase
0,02C451A73FDD41CDD05F1E33E4BC60FC,1,0
1,0D09B305F4E0550B710DFB0ADB496070,3,0
2,0F49D92CEA8C80166B0605ED3D1A8E5D,8,3
3,1C0662D8F63AA9CD3B9DB37506F19BE0,1,0
4,3AD6487EBF165BDE4EECE355B64A80A8,1,0


In [395]:
product_conversion.shape

(30, 3)

In [396]:
product_conversion['conversion'] = (product_conversion['purchase']
                                    / product_conversion['add_to_cart'] * 100)

In [397]:
product_conversion.index = product_conversion['ecom_id']

In [398]:
product_conversion.index

Index(['02C451A73FDD41CDD05F1E33E4BC60FC', '0D09B305F4E0550B710DFB0ADB496070',
       '0F49D92CEA8C80166B0605ED3D1A8E5D', '1C0662D8F63AA9CD3B9DB37506F19BE0',
       '3AD6487EBF165BDE4EECE355B64A80A8', '484FD293DAE723EB16AF9C7548C85ED9',
       '5C8925190BE8C493517153B561BA21E7', '6050B6B1F05ABDB4B33DA4BCEB46DE9B',
       '61F3488FEF9234C5B0E9B68CAF0AB51F', '67FC8BB094EC6205479255C1F8028E20',
       '6D88AAEEAB611F290D931ACCC12045CB', '7D9A98C6E8BF033613B74252D9D24B33',
       '8360E6D158AEBD26BBBE835C38F88748', '9270F8D36E9AB31321F18F04B2F53833',
       '97F9B1135CF24F4CFFF4664435AEC2B9', '9EC302050B63C90533F9FFBCAC489DD8',
       'A6FC1D41C2A36CD6ED1428BC40991F3A', 'AC688D3B8E8D812E2B49496BE59286E3',
       'AD157928CC15ED1D8368385541B01526', 'AF51FBD3B09A5BE324C76C524118BFD3',
       'B0270133170E1B886F4FA77B7E1F0EB9', 'B3A955CA48D147C9C9D7795DC736464D',
       'BDBD23C5359EF7B3F85763C032892B85', 'BEFC96A9D6DEA3A9817BC5E5157B0A1C',
       'BF4304D1BE666DB38DF823DE35823D4D', 'C0C9F2DC

In [399]:
product_conversion.sort_values(by='conversion', ascending=False)

ecom_event_action,ecom_id,add_to_cart,purchase,conversion
ecom_id,,,,
C0C9F2DC21904776545FFBB2FC3D646C,C0C9F2DC21904776545FFBB2FC3D646C,11,11,100.000000
A6FC1D41C2A36CD6ED1428BC40991F3A,A6FC1D41C2A36CD6ED1428BC40991F3A,11,9,81.818182
BF4304D1BE666DB38DF823DE35823D4D,BF4304D1BE666DB38DF823DE35823D4D,9,6,66.666667
6050B6B1F05ABDB4B33DA4BCEB46DE9B,6050B6B1F05ABDB4B33DA4BCEB46DE9B,5,3,60.000000
0F49D92CEA8C80166B0605ED3D1A8E5D,0F49D92CEA8C80166B0605ED3D1A8E5D,8,3,37.500000
C1506A9C4FBD8AD89DA9FA0CC6616566,C1506A9C4FBD8AD89DA9FA0CC6616566,3,1,33.333333
5C8925190BE8C493517153B561BA21E7,5C8925190BE8C493517153B561BA21E7,6,2,33.333333
97F9B1135CF24F4CFFF4664435AEC2B9,97F9B1135CF24F4CFFF4664435AEC2B9,21,7,33.333333
BDBD23C5359EF7B3F85763C032892B85,BDBD23C5359EF7B3F85763C032892B85,12,3,25.000000


In [400]:
# разделим товары по конверсии на 3 группы:
# 1 группа -> конверсия 80% и больше;
# 2 группа -> конверсия 50% и больше;
# 3 группа -> конверсия меньше 50%;

g_con_1 = []
g_con_2 = []
g_con_3 = []

for ecom_id, percent in zip(product_conversion.ecom_id, product_conversion.conversion):
        if percent < 50:
            g_con_3.append(ecom_id)
        elif percent < 80 and percent >= 50:
            g_con_2.append(ecom_id)
        else:
            g_con_1.append(ecom_id)

In [401]:
# здесь также можно давать рекомендации по третьей группе, что и раньше
# но в данных не содержится данные о примерке товара
# это важно, так как для товаров, которые можно примерять
# конверсия 30-40

print('1 группа: ', g_con_1, '\n\n',
      '2 группа: ', g_con_2, '\n\n',
      '3 группа: ', g_con_3, '\n\n')

1 группа:  ['A6FC1D41C2A36CD6ED1428BC40991F3A', 'C0C9F2DC21904776545FFBB2FC3D646C'] 

 2 группа:  ['6050B6B1F05ABDB4B33DA4BCEB46DE9B', 'BF4304D1BE666DB38DF823DE35823D4D'] 

 3 группа:  ['02C451A73FDD41CDD05F1E33E4BC60FC', '0D09B305F4E0550B710DFB0ADB496070', '0F49D92CEA8C80166B0605ED3D1A8E5D', '1C0662D8F63AA9CD3B9DB37506F19BE0', '3AD6487EBF165BDE4EECE355B64A80A8', '484FD293DAE723EB16AF9C7548C85ED9', '5C8925190BE8C493517153B561BA21E7', '61F3488FEF9234C5B0E9B68CAF0AB51F', '67FC8BB094EC6205479255C1F8028E20', '6D88AAEEAB611F290D931ACCC12045CB', '7D9A98C6E8BF033613B74252D9D24B33', '8360E6D158AEBD26BBBE835C38F88748', '9270F8D36E9AB31321F18F04B2F53833', '97F9B1135CF24F4CFFF4664435AEC2B9', '9EC302050B63C90533F9FFBCAC489DD8', 'AC688D3B8E8D812E2B49496BE59286E3', 'AD157928CC15ED1D8368385541B01526', 'AF51FBD3B09A5BE324C76C524118BFD3', 'B0270133170E1B886F4FA77B7E1F0EB9', 'B3A955CA48D147C9C9D7795DC736464D', 'BDBD23C5359EF7B3F85763C032892B85', 'BEFC96A9D6DEA3A9817BC5E5157B0A1C', 'C1506A9C4FBD8AD89DA9F

#### 2.3 Объединим пункты 2.1, 2.2

In [402]:
# выберем необходимые нам столбцы

wild_cut.columns

Index(['Unnamed: 0', 'utc_event_time', 'utc_event_date', 'user_id',
       'city_name', 'ecom_event_action', 'ecom_id', 'ecom_brand',
       'ecom_variant', 'ecom_currency', 'ecom_price', 'ecom_qty',
       'ecom_grand_total', 'os_manufacturer', 'device_type',
       'traffic_src_kind', 'app_version', 'net_type', 'locale'],
      dtype='object')

In [403]:
# сгруппируем изначальную таблицу по id товара, и создадим новые столбцы,
# в которых укажем конверсию, % прибыли, вхождение в группы по конверсии и прибыли

final_table = wild_cut.groupby(['ecom_id', 'ecom_brand', 'ecom_variant',
                                'ecom_currency', 'ecom_price']) \
                      .agg({'user_id': 'count'}) \
                      .rename(columns={'user_id': 'count_produtc'}) \
                      .reset_index(level=['ecom_brand', 'ecom_variant',
                                          'ecom_currency', 'ecom_price'])

In [404]:
final_table.head()

,ecom_brand,ecom_variant,ecom_currency,ecom_price,count_produtc
ecom_id,,,,,
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1800.0,2
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1820.0,1
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1861.0,3
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1882.0,1
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,2045.0,1


In [405]:
# объединим таблицу по столбцу id товара (добавим столбец с % прибыли)

final_table = final_table.merge(our_percent, on='ecom_id', how='left')

In [406]:
# добавляем группу по % прибыли

for ecom_id, row in final_table.iterrows():
        if ecom_id in group_1:
            final_table.at[ecom_id, 'profit_group'] = 'group_1'
        elif ecom_id in group_2:
            final_table.at[ecom_id, 'profit_group'] = 'group_2'
        else:
            final_table.at[ecom_id, 'profit_group'] = 'group_3'

In [407]:
final_table.head()

,ecom_brand,ecom_variant,ecom_currency,ecom_price,count_produtc,profit_percent,profit_group
ecom_id,,,,,,,
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1800.0,2,0.165069,group_3
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1820.0,1,0.165069,group_3
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1861.0,3,0.165069,group_3
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1882.0,1,0.165069,group_3
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,2045.0,1,0.165069,group_3


In [408]:
# объединим таблицу по столбцу id товара (добавим столбец с конверсией)

final_table = final_table.merge(product_conversion[['conversion']],
                                on='ecom_id',
                                how='left')

In [409]:
final_table.head()

,ecom_brand,ecom_variant,ecom_currency,ecom_price,count_produtc,profit_percent,profit_group,conversion
ecom_id,,,,,,,,
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1800.0,2,0.165069,group_3,0.0
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1820.0,1,0.165069,group_3,0.0
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1861.0,3,0.165069,group_3,0.0
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1882.0,1,0.165069,group_3,0.0
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,2045.0,1,0.165069,group_3,0.0


In [410]:
# добавляем группу по конверсии

for ecom_id, row in final_table.iterrows():
        if ecom_id in g_con_1:
            final_table.at[ecom_id, 'conversion_group'] = 'group_1'
        elif ecom_id in g_con_2:
            final_table.at[ecom_id, 'conversion_group'] = 'group_2'
        else:
            final_table.at[ecom_id, 'conversion_group'] = 'group_3'

In [443]:
final_table.head(70)

,ecom_brand,ecom_variant,ecom_currency,ecom_price,count_produtc,profit_percent,profit_group,conversion,conversion_group,price_trigger
ecom_id,,,,,,,,,,
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1800.0,2,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1820.0,1,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1861.0,3,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1882.0,1,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,2045.0,1,0.165069,group_3,0.0,group_3,True
...,...,...,...,...,...,...,...,...,...,...
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1725.0,18,0.941033,group_2,60.0,group_2,True
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1745.0,10,0.941033,group_2,60.0,group_2,True
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1764.0,9,0.941033,group_2,60.0,group_2,True


#### 2.4 Посмотрим какие товары имеют стоимость меньше 500 р

In [418]:
for ecom_id, ecom_price in zip(final_table.index, final_table.ecom_price):
        if ecom_price > 500:
            final_table.loc[ecom_id, 'price_trigger'] = True
        else:
            final_table.loc[ecom_id, 'price_trigger'] = False

In [445]:
final_table.head(70)

,ecom_brand,ecom_variant,ecom_currency,ecom_price,count_produtc,profit_percent,profit_group,conversion,conversion_group,price_trigger
ecom_id,,,,,,,,,,
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1800.0,2,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1820.0,1,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1861.0,3,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,1882.0,1,0.165069,group_3,0.0,group_3,True
02C451A73FDD41CDD05F1E33E4BC60FC,400A5E6FAFF295AFD248524088D84BF2,3B9A6121A509CBADC049AEABD3ED0223,RUB,2045.0,1,0.165069,group_3,0.0,group_3,True
...,...,...,...,...,...,...,...,...,...,...
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1725.0,18,0.941033,group_2,60.0,group_2,True
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1745.0,10,0.941033,group_2,60.0,group_2,True
6050B6B1F05ABDB4B33DA4BCEB46DE9B,4F8BB9BC28218427B1EA3CC1A94E420A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1764.0,9,0.941033,group_2,60.0,group_2,True


### 3 Моделируем опрос для продавца

In [421]:
# представим, что продавец хочет начать продавать какой-то новый товар,
# но чтобы спрогнозировать востребованность товара в будущем, он хочет провести опрос
# среди покупателей

In [441]:
# пусть есть два тестовых товара разных цветов
# мы спрашиваем у покупателя, какой ему вариант больше нравится
# покупатель "голосует" за понравившейся ему товар
# и мы видим данные по голосам

dict_for_research = {'id' :['product_1', 'product_2', 'product_3'],
                     'date': ['2019-06-18', '2019-07-07', '2019-07-16'],
                     'user_id': ['34241asd4', '1rq3441', '1e2fa1212'],
                     'city_name': ['Moscow', 'Tver', 'Irkutsk'],
                     'var_1': [123, 43, 56],
                     'var_2': [23, 90, 51],
                     'total_views': [215, 400, 130]
                     }

research = pd.DataFrame(dict_for_research)

# var_1 -> кол-во пользователей, отдавших голоса за вариант 1
# var_2 -> кол-во пользователей, отдавших голоса за вариант 2
# total_views -> всего просмотров: var_1 + var_2 + отказавшиеся от опроса

In [442]:
research

,id,date,user_id,city_name,var_1,var_2,total_views
0,product_1,2019-06-18,34241asd4,Moscow,123,23,215
1,product_2,2019-07-07,1rq3441,Tver,43,90,400
2,product_3,2019-07-16,1e2fa1212,Irkutsk,56,51,130
